In [11]:
load("core_utils.py")

# First pass - search for values of n that admit a decomposition

In [12]:
existence_values = []

for n in range(3,250):
    # n can have dyadic valuation at most 1 [Theorem 3.4, And+23]
    if n % 4 == 0: continue

    en = (1<<n)-1

    factors = factor(en)
    pm1_divisors = [divisors(p[0]-1) for p in factors]

    found = False

    for i in range(2^dyadic(n)):
        for j in range(1,n//2+1):
            d = (1<<j) + (1<<i)

            # Sieve with the Jacobi Symbol (i.e. even order)
            if jacobi_symbol(d,en) != -1:
                continue

            e = order(d, factors, pm1_divisors)//2
            de = pow(d, e, en)
            if de == en - 1:
                existence_values.append(n)
                break
        
        if found:
            break

existence_values


[3,
 5,
 6,
 7,
 10,
 13,
 14,
 15,
 17,
 19,
 23,
 26,
 30,
 31,
 34,
 38,
 43,
 46,
 51,
 59,
 61,
 62,
 65,
 67,
 79,
 83,
 85,
 86,
 89,
 93,
 95,
 97,
 101,
 102,
 103,
 107,
 109,
 122,
 127,
 129,
 130,
 131,
 133,
 137,
 139,
 149,
 158,
 170,
 178,
 186,
 193,
 197,
 202,
 214,
 215,
 218,
 221,
 227,
 233,
 241]

# Second pass - find shortest decomposition for the values of n that admit a decomposition

In [13]:
try:
    _ = existence_values
except NameError:
    existence_values = [3, 5, 6, 7, 10, 13, 14, 15, 17, 19, 23, 26, 30, 31, 34, 38, 43, 46, 51, 59, 61, 62, 65, 67, 79, 83, 85, 86, 89, 93, 95, 97, 101, 102, 103, 107, 109, 122, 127, 129, 130, 131, 133, 137, 139, 149, 158, 170, 178, 186, 193, 197, 202, 214, 215, 218, 221, 227, 233, 241]

shortest_decompositions = {}

for n in existence_values:
    en = (1<<n)-1

    print(f"{n=} {en=} phi={euler_phi(en)}")

    factors = factor(en)
    pm1_divisors = [divisors(p[0]-1) for p in factors]

    # Just an unreachable upper bound
    shortest_decomposition = (None, euler_phi(en) + 1)

    for i in range(2^dyadic(n)):
        for j in range(1,n//2+1):
            d = (1<<j) + (1<<i)

            # Sieve with the Jacobi Symbol (i.e. even order)
            if jacobi_symbol(d,en) != -1:
                continue

            e = order(d, factors, pm1_divisors)//2
            if e >= shortest_decomposition[1]:
                continue

            de = pow(d, e, en)
            if de == en - 1:
                print(f"\t{d=} {i=} {j=}, {e=}")
                shortest_decomposition = (d, e)
                
    shortest_decompositions[n] = (shortest_decomposition)

    print("")

shortest_decompositions

n=3 en=7 phi=6
	d=3 i=0 j=1, e=3

n=5 en=31 phi=30
	d=3 i=0 j=1, e=15

n=6 en=63 phi=36
	d=5 i=0 j=2, e=3

n=7 en=127 phi=126
	d=3 i=0 j=1, e=63
	d=5 i=0 j=2, e=21

n=10 en=1023 phi=600
	d=17 i=0 j=4, e=15

n=13 en=8191 phi=8190
	d=3 i=0 j=1, e=455
	d=17 i=0 j=4, e=4095
	d=33 i=0 j=5, e=4095

n=14 en=16383 phi=10584
	d=5 i=0 j=2, e=21
	d=65 i=0 j=6, e=63

n=15 en=32767 phi=27000
	d=3 i=0 j=1, e=75

n=17 en=131071 phi=131070
	d=3 i=0 j=1, e=65535
	d=33 i=0 j=5, e=13107
	d=65 i=0 j=6, e=4369
	d=129 i=0 j=7, e=255

n=19 en=524287 phi=524286
	d=3 i=0 j=1, e=262143
	d=5 i=0 j=2, e=37449
	d=17 i=0 j=4, e=262143
	d=65 i=0 j=6, e=262143
	d=257 i=0 j=8, e=262143

n=23 en=8388607 phi=8210080
	d=257 i=0 j=8, e=2231

n=26 en=67108863 phi=44717400
	d=17 i=0 j=4, e=4095
	d=257 i=0 j=8, e=4095
	d=4097 i=0 j=12, e=455

n=30 en=1073741823 phi=534600000
	d=16385 i=0 j=14, e=75

n=31 en=2147483647 phi=2147483646
	d=3 i=0 j=1, e=357913941
	d=5 i=0 j=2, e=97612893
	d=65 i=0 j=6, e=357913941
	d=129 i=0 j=7,

{3: (3, 3),
 5: (3, 15),
 6: (5, 3),
 7: (5, 21),
 10: (17, 15),
 13: (3, 455),
 14: (5, 21),
 15: (3, 75),
 17: (129, 255),
 19: (5, 37449),
 23: (257, 2231),
 26: (4097, 455),
 30: (16385, 75),
 31: (5, 97612893),
 34: (65, 21845),
 38: (262145, 262143),
 43: (65, 197737005),
 46: (257, 135615797),
 51: (4194305, 30001923),
 59: (16385, 305327091563275),
 61: (67108865, 3660068268593165),
 62: (1073741825, 357913941),
 65: (2147483649, 10378224539865),
 67: (33, 15295807610659665),
 79: (5, 576294577078301779),
 83: (2049, 1447815352831891227193),
 85: (2199023255553, 4760486403166879215),
 86: (16777217, 963074800615954005),
 89: (8796093022209, 3641000115545236102644483),
 93: (137438953473, 768614336762478591),
 95: (549755813889, 7354378117756963125),
 97: (17592186044417, 102089228364235081637527141),
 101: (32769, 29053231578384186812775761),
 102: (1125899906842625, 150009615),
 103: (4398046511105, 2051214562671759462384635289),
 107: (67108865, 758220919762679268184943973309